In [1]:
!pip install transformers==3.3.1

     |████████████████████████████████| 1.1MB 11.0MB/s 
     |████████████████████████████████| 890kB 32.3MB/s 
     |████████████████████████████████| 3.0MB 38.6MB/s 
     |████████████████████████████████| 1.1MB 38.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6f8e1514b046345c73c0a46ab91c82b76d2bdd25e4fd5cfccd7e8af7a1d81306
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import pandas as pd
import numpy as np
import re
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW

In [3]:
from tqdm import tqdm, tqdm_notebook

In [4]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

Mounted at /gdrive


In [5]:
cd /gdrive/MyDrive/Colab Notebooks/Mathpresso/

/gdrive/MyDrive/Colab Notebooks/Mathpresso


In [6]:
device = torch.device("cuda:0")

In [7]:
# pandas 설정: 각 column 별 (혹은 한 cell에서) 출력되는 글자수 제한을 없애기
pd.set_option('display.max_colwidth', -1)
df = pd.read_csv('./몽데이크_Open.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [8]:
# 개수 체크하기
print(len(df[df['text']=='다음 중 옳은 것은?']))
print(len(df[df['text']=='다음 중 옳지 않은 것은?']))

10
5


In [9]:
# 15개 밖에 없기 때문에 해도 무방할듯?
df = df[(df['text'] !='다음 중 옳은 것은?') & (df['text'] !='다음 중 옳지 않은 것은?')]

In [10]:
# qtid값 체크
df['qtid'] = df['qtid'].apply(lambda x: x[0:7])

In [11]:
# Math symbols
math_terms = ['sin', 'cos', 'tan', #Triangle function
             'alpha', 'beta', 'theta', 'gamma', 'omega', 'phi', #Roman letters
             'uu', 'nn', 'sup', 'sub', '^C', #Sets
             'pi', 'abs', 'sqrt', 'cdots', #Etc
             '!', '(', ')', '{', '}', '[', ']', '+', '|', '/_', '@', '=' 
             ]

# 좌표 괄호와 일반 괄호 구분 : 추가 예정

# Math symbols which need regular expression
math_terms_complex = [r'_[0-9]+C_[0-9+]', #Combination
                     r'_[0-9]+P_[0-9+]' #Permutation
                     ] 

# Symbols which are not related to math
non_math_terms = ['<br/>', '<fieldset>', '<legend>', '</fieldset>', '</legend>', '\{::\}']

In [12]:
df = df[['text', 'qtid']]

In [13]:
df.reset_index(inplace=True, drop=True)

In [14]:
# 라벨 인코더 생성
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(df['qtid'])
df['qtid'] = encoder.transform(df['qtid'])

In [15]:
'''
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
'''

'\ntrain_size = 0.8\ntrain_dataset=df.sample(frac=train_size,random_state=200)\ntest_dataset=df.drop(train_dataset.index).reset_index(drop=True)\ntrain_dataset = train_dataset.reset_index(drop=True)\n'

In [16]:
train_dataset=df
train_dataset = train_dataset.reset_index(drop=True)

In [17]:
from transformers import ElectraModel, ElectraTokenizer

In [18]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 50
LEARNING_RATE = 1e-05
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [19]:
class ElectraDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        text = str(self.data.text[index])
        text= " ".join(text.split())
        inputs = self.tokenizer(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': self.data.qtid[index]
        } 
    
    def __len__(self):
        return self.len

In [20]:
print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
#print("TEST Dataset: {}".format(test_dataset.shape))

training_set = ElectraDataset(train_dataset, tokenizer, MAX_LEN)
#testing_set = ElectraDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (5349, 2)
TRAIN Dataset: (5349, 2)


In [21]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

#test_params = {'batch_size': VALID_BATCH_SIZE,
#                'shuffle': True,
 #               'num_workers': 0
#                }

training_loader = DataLoader(training_set, **train_params)
#testing_loader = DataLoader(testing_set, **test_params)

In [22]:
class ElectraModelClass(torch.nn.Module):
    def __init__(self):
        super(ElectraModelClass, self).__init__()
        self.l1 = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 37)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [23]:
model = ElectraModelClass()
model.to(device)

ElectraModelClass(
  (l1): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [24]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [25]:
# Function to calcuate the accuracy of the model

def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [26]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [27]:
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 5000 steps: 3.5288734436035156
Training Accuracy per 5000 steps: 0.0
The Total Accuracy for Epoch 0: 14.507384557861283
Training Loss Epoch: 3.336743402997831
Training Accuracy Epoch: 14.507384557861283
Training Loss per 5000 steps: 1.5937796831130981
Training Accuracy per 5000 steps: 100.0
The Total Accuracy for Epoch 1: 39.371845204711164
Training Loss Epoch: 2.4648828040502173
Training Accuracy Epoch: 39.371845204711164
Training Loss per 5000 steps: 2.832660436630249
Training Accuracy per 5000 steps: 0.0
The Total Accuracy for Epoch 2: 64.92802392970648
Training Loss Epoch: 1.5203829385752279
Training Accuracy Epoch: 64.92802392970648
Training Loss per 5000 steps: 0.7296488881111145
Training Accuracy per 5000 steps: 100.0
The Total Accuracy for Epoch 3: 78.29500841278744
Training Loss Epoch: 0.9490484175270893
Training Accuracy Epoch: 78.29500841278744
Training Loss per 5000 steps: 0.6331831216812134
Training Accuracy per 5000 steps: 100.0
The Total Accuracy for Ep

In [28]:
'''
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    predict = []
    answer = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask)
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            predict.append(big_idx[0])
            predict.append(big_idx[1])
            answer.append(targets[0])
            answer.append(targets[1])

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return predict,answer
'''



'\ndef valid(model, testing_loader):\n    model.eval()\n    n_correct = 0; n_wrong = 0; total = 0\n    tr_loss = 0\n    nb_tr_steps = 0\n    nb_tr_examples = 0\n    predict = []\n    answer = []\n    with torch.no_grad():\n        for _, data in enumerate(testing_loader):\n            ids = data[\'ids\'].to(device, dtype = torch.long)\n            mask = data[\'mask\'].to(device, dtype = torch.long)\n            targets = data[\'targets\'].to(device, dtype = torch.long)\n            outputs = model(ids, mask)\n            loss = loss_function(outputs, targets)\n            tr_loss += loss.item()\n            big_val, big_idx = torch.max(outputs.data, dim=1)\n            n_correct += calcuate_accu(big_idx, targets)\n\n            predict.append(big_idx[0])\n            predict.append(big_idx[1])\n            answer.append(targets[0])\n            answer.append(targets[1])\n\n            nb_tr_steps += 1\n            nb_tr_examples+=targets.size(0)\n            \n            if _%5000==0

In [29]:
'''
print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

acc = valid(model, testing_loader)
print(acc[0])
print(acc[1])
'''

"\nprint('This is the validation section to print the accuracy and see how it performs')\nprint('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')\n\nacc = valid(model, testing_loader)\nprint(acc[0])\nprint(acc[1])\n"

In [30]:
'''
pred = []
true = []
for i in range(len(acc[0])):
  pred.append(int(acc[0][i]))
for i in range(len(acc[1])):
  true.append(int(acc[1][i]))
  '''

'\npred = []\ntrue = []\nfor i in range(len(acc[0])):\n  pred.append(int(acc[0][i]))\nfor i in range(len(acc[1])):\n  true.append(int(acc[1][i]))\n  '

In [31]:
'''
pred
'''

'\npred\n'

In [50]:

# Saving the files for re-use

output_model_file = './pytorch_distilbert_news.bin'
output_vocab_file = './vocab_distilbert_news.bin'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')


All files saved
This tutorial is completed


In [58]:
torch.save(model, './model.pt')  # 전체 모델 저장
torch.save(model.state_dict(), './model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, './all.tar')

# test

In [34]:
testset = pd.read_csv('./hidden_for_inference.csv')

# Add spaces to the next of math symbols
for math in math_terms:
    testset['text'] = testset['text'].apply(lambda x: x.replace(math, ' '+math+' '))

# Remove non-math terms
for non_math in non_math_terms:
    testset['text'] = testset['text'].apply(lambda x: x.replace(non_math, ' '))

# Handle exceptions
testset['text'] = testset['text'].apply(lambda x: re.sub(r'_[0-9]+C_[0-9+]', 'Combination', x))
testset['text'] = testset['text'].apply(lambda x: re.sub(r'_[0-9]+P_[0-9+]', 'Permutation', x))
testset['text'] = testset['text'].apply(lambda x: re.sub(r'<보기>', '보기', x))

testset = testset[['text']]
testset.reset_index(inplace=True, drop=True)



In [36]:
class ElectratestDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        text = str(self.data.text[index])
        text= " ".join(text.split())
        inputs = self.tokenizer(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            
        } 
    
    def __len__(self):
        return self.len

In [37]:
test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }
test_set = ElectratestDataset(testset, tokenizer, MAX_LEN)

In [38]:
test_loader = DataLoader(test_set, **test_params)

In [39]:
def testmodel(model, testing_loader):
    model.eval()
    predict = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            outputs = model(ids, mask)
            big_val, big_idx = torch.max(outputs.data, dim=1)

            predict.append(big_idx[0])
            predict.append(big_idx[1])

    return predict

In [40]:
answer = testmodel(model, test_loader)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [41]:
answer

[tensor(9, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(7, device='cuda:0'),
 tensor(8, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(26, device='cuda:0'),
 tensor(4, device='cuda:0'),
 tensor(4, device='cuda:0'),
 tensor(6, device='cuda:0'),
 tensor(5, device='cuda:0'),
 tensor(9, device='cuda:0'),
 tensor(15, device='cuda:0'),
 tensor(11, device='cuda:0'),
 tensor(9, device='cuda:0'),
 tensor(6, device='cuda:0'),
 tensor(6, device='cuda:0'),
 tensor(9, device='cuda:0'),
 tensor(7, device='cuda:0'),
 tensor(20, device='cuda:0'),
 tensor(7, device='cuda:0'),
 tensor(8, device='cuda:0'),
 tensor(9, device='cuda:0'),
 tensor(2, device='cuda:0'),
 tensor(11, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(6, device='cuda:0'),
 tensor(7, device='cuda:0'),
 tensor(20, device='cuda:0'),
 tensor(30, device='cuda:0'),
 tensor(19, device='cuda:0'),
 tensor(25, device='cuda:0'),
 tensor(5, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(10, device='cuda:0'),
 ten

In [42]:
test_pred = []
for i in range(len(answer)):
  test_pred.append(int(answer[i]))

In [43]:
test_pred

[9,
 0,
 7,
 8,
 0,
 26,
 4,
 4,
 6,
 5,
 9,
 15,
 11,
 9,
 6,
 6,
 9,
 7,
 20,
 7,
 8,
 9,
 2,
 11,
 1,
 6,
 7,
 20,
 30,
 19,
 25,
 5,
 0,
 10,
 6,
 8,
 6,
 7,
 10,
 10,
 35,
 3,
 1,
 28,
 3,
 30,
 33,
 7,
 2,
 10,
 31,
 6,
 7,
 1,
 0,
 34,
 26,
 27,
 8,
 20,
 1,
 24,
 30,
 26,
 35,
 6,
 31,
 27,
 24,
 8,
 7,
 22,
 28,
 35,
 35,
 28,
 1,
 8,
 6,
 7,
 11,
 10,
 1,
 2,
 7,
 4,
 36,
 34,
 32,
 32,
 26,
 36,
 24,
 21,
 35,
 33,
 31,
 35,
 33,
 25,
 0,
 9,
 3,
 34,
 30,
 32,
 34,
 25,
 22,
 25,
 7,
 21,
 5,
 34,
 31,
 23,
 31,
 27,
 25,
 31,
 36,
 31,
 27,
 31,
 32,
 36,
 23,
 24,
 29,
 26,
 22,
 20,
 32,
 3,
 3,
 6,
 32,
 31,
 31,
 25,
 24,
 36,
 31,
 34,
 26,
 29,
 27,
 25,
 24,
 22,
 29,
 23,
 25,
 31,
 35,
 26,
 29,
 24,
 21,
 32,
 26,
 25,
 36,
 19,
 35,
 23,
 21,
 21,
 29,
 23,
 3,
 10,
 22,
 7,
 5,
 7,
 8,
 7,
 14,
 15,
 26,
 35,
 32,
 16,
 14,
 5,
 2,
 11,
 9,
 10,
 2,
 4,
 15,
 4,
 6,
 8,
 1,
 30,
 27,
 10,
 8,
 1,
 1,
 16,
 18,
 15,
 31,
 35,
 31,
 33,
 18,
 15,
 18,
 16,
 32,
 

In [44]:
test_pred_encode = encoder.inverse_transform(test_pred)
test_pred_encode[0:6]

array(['H1S1-10', 'H1S1-01', 'H1S1-08', 'H1S1-09', 'H1S1-01', 'HSU1-01'],
      dtype=object)

In [47]:
test_pred_encode

array(['H1S1-10', 'H1S1-01', 'H1S1-08', ..., 'H1S2-07', 'H1S2-06',
       'HSTA-02'], dtype=object)

In [49]:
list(test_pred_encode)

['H1S1-10',
 'H1S1-01',
 'H1S1-08',
 'H1S1-09',
 'H1S1-01',
 'HSU1-01',
 'H1S1-05',
 'H1S1-05',
 'H1S1-07',
 'H1S1-06',
 'H1S1-10',
 'H1S2-04',
 'H1S1-12',
 'H1S1-10',
 'H1S1-07',
 'H1S1-07',
 'H1S1-10',
 'H1S1-08',
 'HSTA-02',
 'H1S1-08',
 'H1S1-09',
 'H1S1-10',
 'H1S1-03',
 'H1S1-12',
 'H1S1-02',
 'H1S1-07',
 'H1S1-08',
 'HSTA-02',
 'HSU1-05',
 'HSTA-01',
 'HSTA-07',
 'H1S1-06',
 'H1S1-01',
 'H1S1-11',
 'H1S1-07',
 'H1S1-09',
 'H1S1-07',
 'H1S1-08',
 'H1S1-11',
 'H1S1-11',
 'HSU1-10',
 'H1S1-04',
 'H1S1-02',
 'HSU1-03',
 'H1S1-04',
 'HSU1-05',
 'HSU1-08',
 'H1S1-08',
 'H1S1-03',
 'H1S1-11',
 'HSU1-06',
 'H1S1-07',
 'H1S1-08',
 'H1S1-02',
 'H1S1-01',
 'HSU1-09',
 'HSU1-01',
 'HSU1-02',
 'H1S1-09',
 'HSTA-02',
 'H1S1-02',
 'HSTA-06',
 'HSU1-05',
 'HSU1-01',
 'HSU1-10',
 'H1S1-07',
 'HSU1-06',
 'HSU1-02',
 'HSTA-06',
 'H1S1-09',
 'H1S1-08',
 'HSTA-04',
 'HSU1-03',
 'HSU1-10',
 'HSU1-10',
 'HSU1-03',
 'H1S1-02',
 'H1S1-09',
 'H1S1-07',
 'H1S1-08',
 'H1S1-12',
 'H1S1-11',
 'H1S1-02',
 'H1

In [55]:
test_answer_df = pd.DataFrame(test_pred_encode, columns={'qtid'})

In [56]:
test_answer_df.to_csv('test_answer_df.csv')

In [46]:
testset_sub = pd.read_csv('./hidden_for_inference.csv')
submission = []
for i in range(len(testset_sub)):
    temp_dic = {}
    temp_dic['qplay_question_id'] = int(testset_sub.loc[i, 'qplay_question_id'])
    temp_dic['predict_category'] = test_pred_encode[i]
    submission.append(temp_dic)

submission[0]

{'predict_category': 'H1S1-10', 'qplay_question_id': 3535}

In [101]:
from pprint import pprint
import requests
import json
HOST = 'http://ec2-13-125-227-119.ap-northeast-2.compute.amazonaws.com:8080/'
TOKEN = "055c2875-d72c-4ea8-9381-a2d1323cb618"
data = {"token" : TOKEN,
        "payload": submission}
# submit your answer 
response = requests.post(f'{HOST}submit/', data = json.dumps(data))
pprint(json.loads(response.text))
pprint(json.loads(requests.get(f'{HOST}rank/').text))

{'message': 'score : 0.8746075767019548 is submitted, you have 2 chances'}
{'message': [{'high_score': 0.0, 'team_name': '고려대학교 KUBIG'},
             {'high_score': 0.8309340694390499,
              'team_name': 'mathpresso_baseline'},
             {'high_score': 0.8746075767019548, 'team_name': '연세대학교 ybigta'},
             {'high_score': 0.8806737399419502, 'team_name': '연세대학교 YAI'}]}


In [ ]:
'''
from pprint import pprint
import requests
import json
HOST = 'http://ec2-13-125-227-119.ap-northeast-2.compute.amazonaws.com:8080/'
TOKEN = "055c2875-d72c-4ea8-9381-a2d1323cb618"
data = {"token" : TOKEN,
        "payload": submission}
# submit your answer 
response = requests.post(f'{HOST}submit/', data = json.dumps(data))
pprint(json.loads(requests.get(f'{HOST}rank/').text))
'''